# Electric System for "Kushu": Jamaican town

## 150 Dwellings layed into five 30-house districts

In [47]:
import pandapower as pp
import pandapower.plotting as pplot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Jamaica Electricity Sector Book of Codes, available from:
http://www.our.org.jm

In [64]:
House_Voltage_Level        = 210e-3 #Three-phase voltage for House voltage level (110 V single phase) as per grid code
Transmission_Voltage_Level = 69     #Transmission levels in Jamaica are 69kV and 138kV. Line close to village is 69kV
Distribution_Voltage_Level = 13.8   #Chosen from grid distribution code options of 24, 13.8, 12, 6.9 and 4 kV levels p.97 

#Lengths extracted from QGIS layout design (km)
House_Distribution_Length             = 0.03
School_and_Court_Distribution_Length  = 0.025
External_Grid_to_Substation_Length    = 1.8
Solar_to_Substation_Length            = 0.7
#12kV line lengths
substation_to_district_length         = [0.185, 0.235, 0.319, 0.402, 0.435, 0.279]

Power_factor = 0.9  #Revised with local grid code transmission voltage drop allowed p.97
Theta = np.arccos(Power_factor)
P2Q = np.tan(Theta)

House_P_Demand = 4500e-6    #MW 1500 kW ADMD per household, 3-house block has 4500 kW demand for phase balancing
House_Q_Demand = House_P_Demand*P2Q

School_and_Court_P_Demand = 2*House_P_Demand
School_and_Court_Q_Demand = School_and_Court_P_Demand*P2Q

R_per_km = 1.203  #Swan cable 210 V ASTM B-232, steel reinforced
x_per_km = 0.382
C_per_km = 9.15
imax_hv_ka = 0.17

R_14OHL  = 0.727  #25mm2 Al XLPE PVC 12 kV conductor
X_14OHL  = 0.052
C_14OHL  = 0.0065   
I_14OHL  = 0.145

R_69OHL = 0.167 #Canton Aluminum Alloy cable 
X_69OHL = 0.052 
C_69OHL = 0.0065   
I_69OHL = 0.145 

In [65]:
net = pp.create_empty_network()
districts = 5
housing_blocks_per_district = 10  #One house per phase -> ten 3-house blocks per district (strip) to balance everything

distribution_transformer_lv = [None]*districts
distribution_transformer_hv = [None]*districts
house_block_bus             = [None]*housing_blocks_per_district

#Solar Generation - renewables.ninja data
df = pd.read_excel ('PV_Values.xlsx')
PV_gen = df.iloc[:,1]

#SUBSTATION DEFINITIONS
#Connection to external grid
Jamaican_grid = pp.create_bus(net, vn_kv = Transmission_Voltage_Level, name = "Jamaican grid")
pp.create_ext_grid(net, bus= Jamaican_grid, vm_pu=1, name='External grid')
    
village_substation_hv    = pp.create_bus(net, vn_kv = Transmission_Voltage_Level, name = "Village substation HV")
village_substation_lv    = pp.create_bus(net, vn_kv = Distribution_Voltage_Level, name = "Village substation LV")


pp.create_line_from_parameters(net,from_bus= Jamaican_grid, to_bus= village_substation_hv, \
                                    length_km=External_Grid_to_Substation_Length, r_ohm_per_km = R_69OHL,\
                                    x_ohm_per_km = X_69OHL,\
                                    c_nf_per_km = X_69OHL, max_i_ka=I_69OHL,\
                                    name = "Jamaican Grid to Village Substation HV")

#Creating solar generation and line to substation
Average_solar_generation = 0.360 #MW during year
Solar_PV_bus = pp.create_bus(net, vn_kv = Transmission_Voltage_Level, name = "Solar Farm")
pp.create_sgen(net, bus = Solar_PV_bus, p_mw = Average_solar_generation, q_mvar = Average_solar_generation*P2Q,\
               name = "Daily solar generation")
pp.create_line_from_parameters(net,from_bus = Solar_PV_bus, to_bus= village_substation_hv, \
                                    length_km=Solar_to_Substation_Length, r_ohm_per_km = R_69OHL,\
                                    x_ohm_per_km = X_69OHL,\
                                    c_nf_per_km = X_69OHL, max_i_ka=I_69OHL,\
                                    name = "Solar Farm to Village Substation HV")   

#Substation transformer from 69 kV to 13.8 kV 
pp.create_transformer_from_parameters(net, hv_bus= village_substation_hv, \
                                      lv_bus= village_substation_lv,\
                                      sn_mva= 0.1, vn_hv_kv= Transmission_Voltage_Level,\
                                      vn_lv_kv= Distribution_Voltage_Level, vkr_percent= 1, vk_percent=1,\
                                      pfe_kw= 0.25, i0_percent= 0.33, name = 'Village Substation HV - LV transformer', \
                                      tap_side = 'hv', tap_max = 10, tap_min = -10, tap_pos = -10, tap_neutral = 0, tap_step_percent = 1)

for district in range(districts):
    #Transformers at top of each district turns voltage down to the house level (210V)
    distribution_transformer_hv[district] = pp.create_bus(net, vn_kv = Distribution_Voltage_Level, \
                                           name = "Distribution transformer hv - District %s" %district)        
    distribution_transformer_lv[district] = pp.create_bus(net, vn_kv = House_Voltage_Level, \
                                           name = "Distribution transformer lv - District %s" %district)        
    pp.create_transformer_from_parameters(net, hv_bus=distribution_transformer_hv[district], \
                                          lv_bus=distribution_transformer_lv[district],\
                                          sn_mva= 0.1, vn_hv_kv= Distribution_Voltage_Level,\
                                          vn_lv_kv= House_Voltage_Level, vkr_percent= 1, vk_percent=1,\
                                          pfe_kw= 0.25, i0_percent= 0.33, name = 'District %s transformer' %district, \
                                          tap_side = 'hv', tap_max = 10, tap_min = -10, tap_pos = -9, tap_neutral = 0, tap_step_percent = 1)
    pp.create_line_from_parameters(net,from_bus= village_substation_lv, to_bus= distribution_transformer_hv[district], \
                                        length_km=substation_to_district_length[district], r_ohm_per_km = R_14OHL,\
                                        x_ohm_per_km = X_14OHL,\
                                        c_nf_per_km = C_14OHL, max_i_ka=I_14OHL,\
                                        name = "Village substation - Distribution transformer %s HV"%district, parallel = 2)

    for housing_block in range(housing_blocks_per_district):
        house_block_bus[housing_block] = pp.create_bus(net, vn_kv = House_Voltage_Level, \
                                         name = "District %s Housing Block %s" %(district, housing_block))
        pp.create_load(net, bus= house_block_bus[housing_block], p_mw=House_P_Demand, q_mvar=House_Q_Demand, \
              name= "District %s House block %s load" %(district, housing_block))

        if housing_block == 0:
            #First three-house block is connected directly to transformer
            pp.create_line_from_parameters(net,from_bus= distribution_transformer_lv[district], to_bus= house_block_bus[housing_block],\
                length_km=House_Distribution_Length, r_ohm_per_km = R_per_km,\
                x_ohm_per_km = x_per_km,c_nf_per_km = C_per_km, max_i_ka=imax_hv_ka,\
                name = "Distribution %s to housing block %s" %(district, housing_block))
        else:
            #Connect houses to each other
            pp.create_line_from_parameters(net,from_bus=house_block_bus[housing_block - 1] ,\
                to_bus = house_block_bus[housing_block],length_km=House_Distribution_Length,\
                r_ohm_per_km = R_per_km, x_ohm_per_km = x_per_km,\
                c_nf_per_km = C_per_km, max_i_ka=imax_hv_ka,\
                name = "District %s Block %s to Block %s" %(district, housing_block - 1, housing_block))  

    
#School and recreation center components
distribution_transformer_SC_hv = pp.create_bus(net, vn_kv = Distribution_Voltage_Level, \
                                               name = "School and Court Distribution transformer HV")        
distribution_transformer_SC_lv = pp.create_bus(net, vn_kv = House_Voltage_Level, \
                                               name = "School and Court Distribution transformer LV")        
pp.create_transformer_from_parameters(net, hv_bus=distribution_transformer_SC_hv, \
                                      lv_bus=distribution_transformer_SC_lv,\
                                      sn_mva= 0.025, vn_hv_kv= Distribution_Voltage_Level,\
                                      vn_lv_kv= House_Voltage_Level, vkr_percent= 1, vk_percent=1,\
                                      pfe_kw= 0.25, i0_percent= 0.33, name = 'School and Court Distribution transformer', \
                                      tap_side = 'hv', tap_max = 10, tap_min = -10, tap_pos = -10, tap_neutral = 0, tap_step_percent = 1)
pp.create_line_from_parameters(net,from_bus= village_substation_lv, to_bus= distribution_transformer_SC_hv, \
                                    length_km=substation_to_district_length[5], r_ohm_per_km = R_14OHL,\
                                    x_ohm_per_km = X_14OHL,\
                                    c_nf_per_km = C_14OHL, max_i_ka=I_14OHL,\
                                    name = "Village substation - School and Court Distribution transformer")

School_and_Court_bus = pp.create_bus(net, vn_kv = House_Voltage_Level, name = "School and Court bus")
pp.create_load(net, bus= School_and_Court_bus, p_mw=School_and_Court_P_Demand, q_mvar=School_and_Court_Q_Demand, \
               name= "School and Court load")

pp.create_line_from_parameters(net,from_bus= distribution_transformer_SC_hv, to_bus= School_and_Court_bus, \
                                    length_km=School_and_Court_Distribution_Length, r_ohm_per_km = R_per_km,\
                                    x_ohm_per_km = x_per_km,\
                                    c_nf_per_km = C_per_km, max_i_ka=imax_hv_ka,\
                                    name = "School and Court Distribution transformer - School and Court")

pp.runpp(net);

In [66]:
net.res_bus

,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,0.074793,0.046859
1,1.000006,-0.000085,0.000000,0.000000
2,1.085524,0.605875,0.000000,0.000000
3,1.000016,-0.000173,-0.360000,-0.174356
4,1.085506,0.606232,0.000000,0.000000
...,...,...,...,...
62,0.956273,2.373533,0.004500,0.002179
63,0.951809,2.412196,0.004500,0.002179
64,1.085514,0.606065,0.000000,0.000000
65,1.206067,0.606065,0.000000,0.000000


In [67]:
#net.bus
net.res_line

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,-0.074793,-0.046859,0.074794,0.046719,4.914178e-07,-1.398464e-04,0.000739,0.000738,0.000739,1.000000,0.000000,1.000006,-0.000085,0.509314
1,0.360000,0.174356,-0.359996,-0.174409,3.928689e-06,-5.322179e-05,0.003347,0.003347,0.003347,1.000016,-0.000173,1.000006,-0.000085,2.308347
2,0.053527,0.024617,-0.053525,-0.024618,1.040179e-06,-9.514807e-08,0.002271,0.002271,0.002271,1.085524,0.605875,1.085506,0.606232,0.782992
3,0.052929,0.024312,-0.050963,-0.023688,1.965876e-03,6.242381e-04,0.134749,0.134749,0.134749,1.188393,0.705113,1.146640,0.963366,79.263838
4,0.046463,0.021509,-0.044831,-0.020990,1.631677e-03,5.181171e-04,0.122762,0.122762,0.122762,1.146640,0.963366,1.108615,1.212521,72.212792
5,0.040331,0.018811,-0.039013,-0.018392,1.318735e-03,4.187458e-04,0.110363,0.110363,0.110363,1.108615,1.212521,1.074442,1.448873,64.919582
6,0.034513,0.016213,-0.033482,-0.015885,1.030721e-03,3.272905e-04,0.097570,0.097570,0.097570,1.074442,1.448873,1.044241,1.668430,57.394157
7,0.028982,0.013706,-0.028211,-0.013461,7.713653e-04,2.449349e-04,0.084407,0.084407,0.084407,1.044241,1.668430,1.018124,1.867032,49.650895
8,0.023711,0.011282,-0.023166,-0.011109,5.443304e-04,1.728425e-04,0.070905,0.070905,0.070905,1.018124,1.867032,0.996190,2.040516,41.708860
9,0.018666,0.008929,-0.018313,-0.008817,3.530811e-04,1.121135e-04,0.057106,0.057106,0.057106,0.996190,2.040516,0.978529,2.184919,33.591875


In [68]:
#line_loading = net.res_line.iloc[:,13]
#line_loading

In [53]:
#net.line

In [46]:
#fig, ax = plt.subplots()
#plt.gcf().set_size_inches(20,12)
#ax.plot(line_loading.values)
#
#   
#ax.legend(loc = 1, framealpha = 1, bbox_to_anchor=(1.18, 1))
#fig.suptitle("Line loading", fontsize=22)
#ax.set_xlabel('Line', fontsize = 20)
#ax.set_ylabel('Line loading %', fontsize= 20)
#
#ax.get_xaxis().tick_bottom()
#ax.get_yaxis().tick_left()